In [16]:
import os
import numpy as np
import pandas as pd
from collections import Counter
import re
import spacy
nlp = spacy.load('en_core_web_sm')
from pathlib import Path

import json


In [17]:
path =[PATH]
direc=[]
for root, directories, files in os.walk(path):
    for directory in directories:
        direc.append(os.path.join(root, directory))
#open the directory and save files into a list too
file_sub=[]
for i in direc:
    path=i
    for files in os.walk(path):
        for file in files:
            file_sub.append(file)
        #filter out empty str and unneeded info
        file_sub=list(filter(None, file_sub))
file_sub=file_sub[1::2]

files_list=[]
for i in direc:
    path=i
    for root, dirs, files in os.walk(path):
        for name in files:
            if name.endswith((".json")):
                full_path = os.path.join(root, name)
                files_list.append(full_path)

uniquefiles=[]
for i in files_list:
    if ".json" in str(i):
        file=re.findall(r"^(.*?)\.json", i)[0]
        uniquefiles.append(file)

# print(uniquefiles)
# for i in uniquefiles:
#     #open key level dataset
#     print(i)
#     df=pd.read_csv(i+"_keylevel.csv")

In [ ]:
path =[PATH]
direc=[]
for root, directories, files in os.walk(path):
    for directory in directories:
        direc.append(os.path.join(root, directory))
#open the directory and save files into a list too
file_sub=[]
for i in direc:
    path=i
    for files in os.walk(path):
        for file in files:
            file_sub.append(file)
        #filter out empty str and unneeded info
        file_sub=list(filter(None, file_sub))
file_sub=file_sub[1::2]

files_list=[]
for i in direc:
    path=i
    for root, dirs, files in os.walk(path):
        for name in files:
            if name.endswith((".json")):
                full_path = os.path.join(root, name)
                files_list.append(full_path)

uniquefiles=[]
for i in files_list:
    if ".json" in str(i):
        file=re.findall(r"^(.*?)\.json", i)[0]
        uniquefiles.append(file)

import pandas as pd

def timestamps(df):
    timepress_start=[df.loc[0, "timepress"]]
    timepress_end=[]
    chunkid=0
    current_timepress=df.loc[0, "timepress"]
    current_session=df.loc[0,"session"]

    for i in range(df["keyname"].shape[0]):
        if (df.loc[i, "chunkid"] != chunkid):
            if current_session != df.loc[i,"session"]:
                timepress_end.append(df.loc[i-2,"timepress"])
                current_session=df.loc[i,"session"]
            else:
                timepress_end.append(current_timepress)
            timepress_start.append(df.loc[i, "timepress"])
            current_timepress=df.loc[i, "timepress"]
            chunkid +=1
        current_timepress=df.loc[i, "timepress"]
        if i ==(len(df)-1):
            timepress_end.append(df.loc[i,"timepress"])

    return timepress_start, timepress_end

def recup(df):
    current_chunkid=0
    session=[]
    userid=[]
    quizid=[]
    chunk=[]
    chunkpos=[]
    burst_type=[]
    
    for i in df.index:
        if (i==0):
            userid.append(df.loc[i,"userid"])
            session.append(df.loc[i,"session"])
            # quizid.append(df.loc[i,"quizid"])
            chunk.append(df.loc[i,"chunk"])
            burst_type.append(df.loc[i,"burst_type"])

        if (df.loc[i, "chunkid"] != current_chunkid)and i>0:
            userid.append(df.loc[i,"userid"])
            session.append(df.loc[i,"session"])
            # quizid.append(df.loc[i,"quizid"])
            chunk.append(df.loc[i,"chunk"])
            burst_type.append(df.loc[i,"burst_type"])
            current_chunkid+=1  
        
    return session,userid,quizid,chunk,burst_type

##############################################

for filepath in uniquefiles:
    #open key level dataset
    
    df=pd.read_csv(filepath+"_keylevel.csv")


    timepress_start, timepress_end = timestamps(df)

    #part of speech of the words
    pos = []
    zero=["/"]
    for doc in nlp.pipe(df['word'].astype('unicode').values):
        if doc.is_parsed:
            pos.append([n.pos_ for n in doc])
        else:
            pos.append(None)
        if (pos[-1] == []):
            pos[-1] = zero
        
    pos_new = [i[0] for i in pos]
    df['word_pos'] = pos_new

    session,userid,quizid,chunk,burst_type=recup(df)
    #concatenate the lists into a column

    dfw = pd.DataFrame(list(zip(userid, session, chunk, timepress_start, timepress_end,burst_type)),
                columns =['userid','session','chunk', 'timepress_start', 'timepress_end','burst_type'])


    # todrop=[""]
    # dfw = dfw[~dfw['word'].isin(todrop)]
    dfw =dfw.reset_index()
    dfw
    df1=df
    df=dfw

    def pause_1(df):
        df["pause_1_after"]=None
        count_row = (df.shape[0]) -2 
        current_session=df.loc[0,"session"]
        for i in range(df["chunk"].shape[0]):
            if i <=count_row:
                x=df.loc[i,"timepress_end"]
                y=df.loc[i+1,"timepress_start"]
                z=y-x
                if df.loc[i+1,"session"] != current_session:
                    z=None
                    current_session=df.loc[i+1,"session"]
                df.loc[i,"pause_1_after"]=z
        df["pause_1_before"] = None
        df.iloc[1:, df.columns.get_loc("pause_1_before")] = df.iloc[:-1, df.columns.get_loc("pause_1_after")].values
            
    pause_1(df)

    def duration(df):
        df["duration"]=0
        count_row = (df.shape[0]) -1 
        for i in range(df["chunk"].shape[0]):
            if i <=count_row:
                x=df.loc[i,"timepress_end"]
                y=df.loc[i,"timepress_start"]
                z=x-y
                if z<0:
                    z= None
                    
                df.loc[i,"duration"]=z
    duration(df)

    def nchar(df):
        df["nchar_chunk"]=0
    #     df["nchar_sen"]=0
    #     df["nchar_session"]=0
        for i in range(df["chunk"].shape[0]):
            count=0
            for y in str(df.loc[i,"chunk"]):
                if y in [ "[","]"]:
                    count+=1
            df.loc[i,"nchar_chunk"]=len(str(df.loc[i,"chunk"]))-count

                    
            if df.loc[i,"burst_type"]=="r-burst":
                df.loc[i,"nchar_chunk"]=df.loc[i,"nchar_chunk"]-2
    #         df.loc[i,"nchar_sentence"]=len(df.loc[i,"sentence"])
    #         df.loc[i,"nchar_session"]=len(df.loc[i,"session_text"])

    nchar(df)


    def char_speed_chunk(df):
        df["char_speed_chunk"]=0
        for i in range(df["chunk"].shape[0]):
            if df.loc[i,"duration"] != 0:
                df.loc[i,"char_speed_chunk"]=((df.loc[i,"nchar_chunk"])/(df.loc[i,"duration"]))*1000
    char_speed_chunk(df)

    from nltk.corpus import words

    pos = []
    zero=["/"]

    df["left_boundary_pos"]=0
    df["right_boundary_pos"]=0

    for doc in nlp.pipe(df['chunk'].astype('unicode').values):
        if doc.is_parsed:
            pos.append([n.pos_ for n in doc])
        else:
            pos.append(None)
        if (pos[-1] == []):
            pos[-1] = zero
            
        
        
    for i in range(len(pos)):
        df.loc[i,"left_boundary_pos"]=pos[i][0]
        df.loc[i,"right_boundary_pos"]=pos[i][-1]


    def chunk_boundary(df):
        df["left_boundary_chunk"]=0
        df['left_boundary_chunk'] = df['chunk'].str.split().str[0].str.strip()
        df["right_boundary_chunk"]=0
        df['right_boundary_chunk'] = df['chunk'].str.split().str[-1].str.strip()
    #     df["left_boundary_pos"]=0
    #     df['left_boundary_pos'] = df['chunkpos'].str.split().str[0].str.strip()
    #     df["right_boundary_pos"]=0
    #     df['right_boundary_pos'] = df['chunkpos'].str.split().str[-1].str.strip()

    chunk_boundary(df)


    suffix=["eer","er","ion","ity","ment","ness","or","sion","ship","th","able","ible","al","ant","ary","ful","ic","ious", "ous", "ive", "less","y","ed","en","er","ing","ize","ise","ly","ward","wise","s"]
    df["left_suffix"]=0
    for i in df.index:
        if df.loc[i,"left_boundary_chunk"] in suffix:
            df.loc[i,"left_suffix"]=1


    from nltk.corpus import words
    import re

    df["incomplete_right"]=0
    df["incomplete_left"]=0
    for i in df.index:
        if i < len(df)-1:
            if df.loc[i,"burst_type"]=="p-burst":
                x=str(df.loc[i,"right_boundary_chunk"])+str(df.loc[i+1,"left_boundary_chunk"])
                if x.lower() in words.words():
                    df.loc[i,"incomplete_right"]=1
                    df.loc[i+1,"incomplete_left"]=1
                    x=""
            if i<len(df)-2 and df.loc[i,"burst_type"]=="r-burst":
                y=str(df.loc[i,"right_boundary_chunk"])
                y=re.sub(r'[^\w]', ' ', y)
                x=y+str(df.loc[i+2,"left_boundary_chunk"])
                if x.lower() in words.words():
                    df.loc[i,"incomplete_right"]=1
                    df.loc[i+2,"incomplete_left"]=1
                    x=""
    punct=['""',"''",'.',',','?','!',':',';',"-"] #à completer
    df["punct_right"]=0
    df["punct_left"]=0
    for i in df.index:
        if df.loc[i,"right_boundary_chunk"] in punct:
            df.loc[i,"punct_right"]=1
        if df.loc[i,"left_boundary_chunk"] in punct:
            df.loc[i,"punct_left"]=1


    df["burst_type_before"]=0
    df.iloc[1:, df.columns.get_loc("burst_type_before")] = df.iloc[:-1, df.columns.get_loc("burst_type")].values



    df["right_boundary_summary"]=0
    df["left_boundary_summary"]=0
    for i in df.index:
        if df.loc[i,"right_boundary_pos"]=="SPACE" :
            df.loc[i,"right_boundary_summary"]=df.loc[i,"right_boundary_pos"]
        elif df.loc[i,"incomplete_right"]==1:
            df.loc[i,"right_boundary_summary"]="INCOMPLETE"
        elif df.loc[i,"punct_right"]==1:
            df.loc[i,"right_boundary_summary"]="PUNCT"    
        else:
            df.loc[i,"right_boundary_summary"]="WORD"
        if df.loc[i,"left_boundary_pos"]=="SPACE": 
            df.loc[i,"left_boundary_summary"]=df.loc[i,"left_boundary_pos"]
        elif df.loc[i,"incomplete_left"]==1:
            df.loc[i,"left_boundary_summary"]="INCOMPLETE"
        elif df.loc[i,"punct_left"]==1:
            df.loc[i,"left_boundary_summary"]="PUNCT"    
        else:
            df.loc[i,"left_boundary_summary"]="WORD"

    df["chunk_before"]=0
    df.iloc[1:, df.columns.get_loc("chunk_before")] = df.iloc[:-1, df.columns.get_loc("chunk")].values
    df["chunk_after"]=0
    df.iloc[:-1, df.columns.get_loc("chunk_after")] = df.iloc[1:, df.columns.get_loc("chunk")].values
                
    # ## add the sentence pos
    pos = []
    for doc in nlp.pipe(df['chunk'].astype('unicode').values):
        if doc.is_parsed:
            pos.append([n.pos_ for n in doc])
                    
            for i in range(len(pos)):
                if df.loc[i,"incomplete_right"]==1:
                    pos[i][-1]="INCOMPLETE"
                if df.loc[i,"incomplete_left"]==1  :
                    pos[i][0]= "INCOMPLETE"
        else:
            pos.append(None)
        if (pos[-1] == []):
            pos[-1] = zero
    df['chunkpos'] = pos

    def split(word): 
        return [char for char in word] 

    def flag_pos_chunk(df):
        df["VERB_in_chunk"]=0
        df["NOUN_in_chunk"]=0
        df["ADJ_in_chunk"]=0
        df["ADV_in_chunk"]=0
        df["PRON_in_chunk"]=0
        for i in df.index:
            if any(ext in  str(df.loc[i,'chunkpos']) for ext in ['VERB']):
                df.loc[i,"VERB_in_chunk"]=1
            if any(ext in  str(df.loc[i,'chunkpos']) for ext in ['NOUN']):
                df.loc[i,"NOUN_in_chunk"]=1
            if any(ext in  str(df.loc[i,'chunkpos']) for ext in ['ADJ']):
                df.loc[i,"ADJ_in_chunk"]=1
            if any(ext in  str(df.loc[i,'chunkpos']) for ext in ['ADV']):
                df.loc[i,"ADV_in_chunk"]=1            
            if any(ext in  str(df.loc[i,'chunkpos']) for ext in ['PRON']):
                df.loc[i,"PRON_in_chunk"]=1              



    flag_pos_chunk(df)       

    def cleanup(df):
        for i in df.index:
            df.loc[i,"right_boundary_chunk"]=str(df.loc[i,"right_boundary_chunk"]).replace("[","")
            df.loc[i,"right_boundary_chunk"]=str(df.loc[i,"right_boundary_chunk"]).replace("]","")
            df.loc[i,"left_boundary_chunk"]=str(df.loc[i,"left_boundary_chunk"]).replace("[","")
            df.loc[i,"left_boundary_chunk"]=str(df.loc[i,"left_boundary_chunk"]).replace("]","")
            df.loc[i,"left_boundary_chunk"]=str(df.loc[i,"left_boundary_chunk"]).replace("nan","")
            df.loc[i,"right_boundary_chunk"]=str(df.loc[i,"right_boundary_chunk"]).replace("nan","")


    cleanup(df)


    punct=['""',"''",'.',',','?','!',':',';',"-"] #à completer

    def punctuation_side(df):
        df["punct_right"]=0
        df["punct_left"]=0
        df["first_char_punct"]=0
        df["last_char_punct"]=0
        for i in df.index:
    # 

            if df.loc[i,"right_boundary_chunk"] in punct:
                df.loc[i,"punct_right"]=1
                    
            if df.loc[i,"left_boundary_chunk"] in punct:
                df.loc[i,"punct_left"]=1
                    
            if len(str(df.loc[i,"right_boundary_chunk"])) > 0 :
                if str(df.loc[i,"right_boundary_chunk"])[-1] in punct:
                    df.loc[i,"last_char_punct"]=str(df.loc[i,"right_boundary_chunk"])[-1]
                    df.loc[i,"punct_right"]=1
                    
                    
            if len(str(df.loc[i,"left_boundary_chunk"])) > 0:
                if str(df.loc[i,"left_boundary_chunk"])[0] in punct:
                    df.loc[i,"first_char_punct"]=str(df.loc[i,"left_boundary_chunk"])[0]
                    df.loc[i,"punct_left"]=1

    punctuation_side(df)

    def summary_chunk_boundary(df):
        df["right_boundary_summary"]=0
        df["left_boundary_summary"]=0
        for i in df.index:
            if df.loc[i,"right_boundary_pos"]=="SPACE" :
                df.loc[i,"right_boundary_summary"]=df.loc[i,"right_boundary_pos"]
            elif df.loc[i,"incomplete_right"]==1:
                df.loc[i,"right_boundary_summary"]="INCOMPLETE"
            elif df.loc[i,"punct_right"]==1:
                df.loc[i,"right_boundary_summary"]="PUNCT"    
            else:
                df.loc[i,"right_boundary_summary"]="WORD"
            if df.loc[i,"left_boundary_pos"]=="SPACE": 
                df.loc[i,"left_boundary_summary"]=df.loc[i,"left_boundary_pos"]
            elif df.loc[i,"incomplete_left"]==1:
                df.loc[i,"left_boundary_summary"]="INCOMPLETE"
            elif df.loc[i,"punct_left"]==1:
                df.loc[i,"left_boundary_summary"]="PUNCT"    
            else:
                df.loc[i,"left_boundary_summary"]="WORD"

    # summary_chunk_boundary(dfC)
    summary_chunk_boundary(df)


    def split(word): 
        return [char for char in word]  
    ##First step : subset by user

    #make a list of punctuation symbols :
    punct=['""',"''",'.',',','?','!',':',';'] 

    def punct_middle(dfC):
        dfC["punct_middle"]=0
        dfC["punct_amount"]=0

        for i in dfC.index:
            counter=0
            real_count=0
            if any(ext in  split(str(dfC.loc[i,'chunk'])) for ext in punct):
                for j in split(str(dfC.loc[i,'chunk'])):
                    if j in punct:
                        real_count+=1

                    #update real count

                #Check if the punctuation mark is at the boundary of the chunk
                    if dfC.loc[i,"punct_right"]==1:
                        counter+=1
                    if dfC.loc[i,"punct_left"]==1:
                        counter+=1
                    if real_count > counter:
                        dfC.loc[i,"punct_middle"]=1
                    
                    dfC.loc[i,"punct_amount"]= real_count
                    
    punct_middle(df)

    def punct_position(df):
        df["punct_position"]=""
        for i in df.index :
            if df.loc[i,"punct_left"]==1:
                df.loc[i,"punct_position"]="left"
            if df.loc[i,"punct_middle"]==1:
                df.loc[i,"punct_position"]="middle"
            if df.loc[i,"punct_right"]==1:
                df.loc[i,"punct_position"]="right"
                
    punct_position(df)

    def chunk_word_count(df):
        df["word_count"]=""
        for i in df.index:
            df.loc[i,"word_count"] = len(str(df.loc[i,"chunk"]).split()) 
            

    chunk_word_count(df)
    def burst_type_count(df):
        df["p_burst_index"]=0
        df["r_burst_index"]=0
        for i in df.index:
            if df.loc[i,"burst_type"]=="p-burst":
                    df.loc[i,"p_burst_index"]=1
            if df.loc[i,"burst_type"]=="r-burst":
                    df.loc[i,"r_burst_index"]=1
                
    burst_type_count(df)

    df.to_csv(str(filepath)+'_chunklevel.csv',index=False,encoding="utf-8")

    df1=df
    text=""
    chunks=[]
    pause=[]
    current_chunk=""
    chunks.append(df1.loc[0,"chunk"])
    chunkid=df1.loc[0,"index"]
    for i in df1.index:
        if df1.loc[i,"index"] != chunkid:
            if str(df1.loc[i,"chunk"])=="nan":
                chunks.append("[backspace]")
            else:
                chunks.append(df1.loc[i,"chunk"])
            pause.append(df1.loc[i,"pause_1_before"])
            chunkid +=1
    counter=0
    for i in range(len(chunks)-1):
        a=str(chunks[i])
        b=str(pause[i])
        #with numbers
        # c= str(counter)+" - "+a + " "+("["+ b +"]").rjust(10)+"\n"
        #without numbers
        c= a + " "+("["+ b +"]").rjust(10)+"\n"

        # print(c, sep=' ', end='', flush=True)
        text=text+c
        counter+=1
    # print(str(counter)+" - "+str(chunks[-1]))
    # text=text + c
    # print(text)

    textfile = open(str(filepath)+"-copie_chunk"+'.txt', 'w')
    textfile.write(text)
    textfile.close()
